In [1]:
import torch

In [3]:
X,y = torch.load('alsaad_phase1.pt',weights_only=True)

In [ ]:
X.shape,y.shape

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [ ]:
from sage.models import ResNetv2,ResBlockv2

hyperparameters = {
    'block':ResBlockv2,
    'widthi':[4],
    'depthi':[1],
    'n_output_neurons':1,
    'norm':'layer'
}
model = ResNetv2(**hyperparameters)
optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4)
criterion = torch.nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)

In [30]:
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset,DataLoader

In [49]:
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=32)
testloader = DataLoader(TensorDataset(X_test,y_test),batch_size=32)
trainlossi = []
testlossi = []
trainf1i = []
testf1i = []

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
from sklearn.metrics import f1_score

for epoch in range(100):
    running_loss = 0.0
    train_preds, train_labels = [], []

    for i, (Xi, yi) in enumerate(trainloader):
        Xi, yi = Xi.to(device), yi.to(device)
        optimizer.zero_grad()
        logits = model(Xi)
        loss = criterion(logits, yi)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        train_preds.extend(torch.sigmoid(logits).round().detach().cpu().numpy())
        train_labels.extend(yi.cpu().numpy())

    trainlossi.append(running_loss / len(trainloader))

    running_loss = 0.0
    test_preds, test_labels = [], []

    with torch.no_grad():
        running_loss = 0.0
        for i, (Xi, yi) in enumerate(testloader):
            Xi, yi = Xi.to(device), yi.to(device)
            logits = model(Xi)
            loss = criterion(logits, yi)
            running_loss += loss.item()

            test_preds.extend(torch.sigmoid(logits).round().cpu().numpy())
            test_labels.extend(yi.cpu().numpy())

        testlossi.append(running_loss / len(testloader))
    train_f1 = f1_score(train_labels, train_preds, average='macro')
    test_f1 = f1_score(test_labels, test_preds, average='macro')
    trainf1i.append(train_f1)
    testf1i.append(test_f1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.plot(trainlossi)
    ax1.plot(testlossi)
    ax2.plot(trainf1i)
    ax2.plot(testf1i)
    plt.savefig('loss.jpg')
    plt.close()

In [ ]:
from sage.utils import evaluate_sigmoid
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

loss,y_true,y_pred = evaluate_sigmoid(trainloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))
loss,y_true,y_pred = evaluate_sigmoid(testloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))

In [ ]:
X,y = torch.load('iftakhar_phase2.pt',weights_only=True)
evalloader = DataLoader(TensorDataset(X,y),batch_size=32)
loss,y_true,y_pred = evaluate_sigmoid(evalloader,model,criterion)
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
print(classification_report(y_true,y_pred))